In [1]:
# default_exp capsAnalaysis

In [2]:
import sys

In [3]:
sys.path.append('/workspace/oct_ca_seg/oct/')

In [4]:
from oct.startup import *
from model import CapsNet
import numpy as np
import mlflow
from fastai.vision import *
import mlflow.pytorch as MLPY
from fastai.utils.mem import gpu_mem_get_all

In [5]:
gpu_mem_get_all()

[GPUMemory(total=16280, free=15401, used=879)]

In [6]:
cocodata_path = Path('/workspace/oct_ca_seg/COCOdata/')
train_path = cocodata_path/'train/images'
valid_path = cocodata_path/'valid/images'
test_path = cocodata_path/'test/images'

In [7]:
metrics = [sens, spec, dice, my_Dice_Loss, acc]

In [8]:
runsave_dir = Path('/workspace/oct_ca_seg/runsaves/')

In [9]:
run_name = 'DEEPCAP_0001_30_learnersaved'

In [10]:
run_name = 'DEEPCAP_0001_30_small_learnersaved'

In [11]:
run_dir = runsave_dir/run_name

In [12]:
cfg_file = 'configCAPS_APPresnet18_bs16_epochs15_lr0.001.json'

In [13]:
cfg_file = 'configCAPS_small_bs24_epochs30_lr0.001.json'

In [14]:
def loadConfigRun(run_dir, name):
    with open(run_dir/name, 'r') as file:
        return json.load(file)

In [15]:
cfg_dict = loadConfigRun(run_dir, cfg_file)

In [16]:
config = DeepConfig(cfg_dict)

## Train, Valid

In [17]:
fn_get_y = lambda image_name: Path(image_name).parent.parent/('labels/'+Path(image_name).name)
codes = np.loadtxt(cocodata_path/'codes.txt', dtype=str)
tfms = get_transforms()
src = (SegCustomItemList
       .from_folder(cocodata_path, recurse=True, extensions='.jpg')
       .filter_by_func(lambda fname: Path(fname).parent.name == 'images', )
       .split_by_folder('train', 'valid')
       .label_from_func(fn_get_y, classes=codes))
src.transform(tfms, tfm_y=True, size=config.LEARNER.img_size)
data = src.databunch(cocodata_path,
                     bs=config.LEARNER.bs,
                     val_bs=2*config.LEARNER.bs,
                     num_workers = config.LEARNER.num_workers)
stats = [torch.tensor([0.2190, 0.1984, 0.1928]), torch.tensor([0.0645, 0.0473, 0.0434])]
data.normalize(stats);
data.c_in, data.c_out = 3, 2

## Test

In [ ]:
fn_get_y = lambda image_name: Path(image_name).parent.parent/('labels/'+Path(image_name).name)
codes = np.loadtxt(cocodata_path/'codes.txt', dtype=str)
tfms = get_transforms()
src = (SegCustomItemList
       .from_folder(test_path, recurse=True, extensions='.jpg')
       .filter_by_func(lambda fname: Path(fname).parent.name == 'images', )
       .split_by_rand_pct(1.0)
       .label_from_func(fn_get_y, classes=codes))
src.transform(tfms, tfm_y=True, size =config.LEARNER.img_size)
data = src.databunch(test_path,
                     bs=config.LEARNER.bs,
                     val_bs=2*config.LEARNER.bs,
                     num_workers = config.LEARNER.num_workers)
stats = [torch.tensor([0.2190, 0.1984, 0.1928]), torch.tensor([0.0645, 0.0473, 0.0434])]
data.normalize(stats);
data.c_in, data.c_out = 3, 2

In [18]:
deepCap = CapsNet(config.MODEL).cuda()
learn = Learner(data = data,
                  model = deepCap,
                  metrics = metrics)

In [ ]:
learn = learn.load(run_dir/'learner')

In [ ]:
class Analysis():
    def __init__(self, learner:Learner, run_dir:Path):
        self.learner = learner
        self.run_dir = run_dir
        self.metrics = pd.read_json(self.run_dir/'metrics.json')
        self.trainL =  pd.read_json(self.run_dir/'trainL.json')
        self.validL =  pd.read_json(self.run_dir/'validL.json')
    
    def remove0FromMetric(self, metric):
        zero_dict = {k:v for (k,v) in self.results[metric].items() if v==0}
        zero_dict = {k: v for k, v in sorted(zero_dict.items(), key=lambda item: item[1])}
        new_dict = {k:v for (k,v) in self.results[metric].items() if v!=0}
        new_dict = {k: v for k, v in sorted(new_dict.items(), key=lambda item: item[1])}
        print('there were ', len(zero_dict.keys()), 'zeros in the ', metric,' list.')
        return zero_dict, new_dict
    
    def histPlotMetric(self, metric, save=False):
        #metric = one of 'dices', 'sens', 'specs', 'accs' 
        zeros, actual = self.remove0FromMetric(metric)
        l_actual = list(actual.values())
        l_actual = [x for x in l_actual if x==x] # get rid of nans
        mean, std, median, max, min = np.mean(l_actual), np.std(l_actual), np.median(l_actual), np.max(l_actual), np.min(l_actual)
        print(mean, std, median, max, min)
        plt.hist(l_actual)
        if save: plt.savefig(self.checkpoint_path/(metric+'.png'))
        #return l_actual
    
    def showPrediction(self, id):
        d = validCOCO.imgs[id]
        im = cv2.imread(d["file_name"])
        print(id, Path(d["file_name"]).name)

        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                       metadata=valid_metadata, 
                       scale=0.8, 
                       instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
                      )

        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        fig, (ax1, ax2) = plt.subplots(1,2)
        image_plot = ax1.imshow(v.get_image()[:, :, ::-1])
        
        label_full = annsToSingleBinMask(validCOCO, d['image_id']) 
        dice_loss = lossdice(self.processOutputsToMask(outputs).cpu(), torch.tensor(label_full)).item()
        label_plot = ax2.imshow(label_full)
        print(dice_loss)

In [ ]:
preds = learn.model(data.one_batch(ds_type=DatasetType.Valid)[0].cuda()).detach().cpu()

In [ ]:
CapsAnalysis = Analysis(learn, run_dir)

In [ ]:
CapsAnalysis.trainL.sort_index().plot.line()

In [ ]:
CapsAnalysis.metrics['dice'].sort_index().plot.line()

In [ ]:
def freeze_to(learn, n): # freeze model up to layer n
    for i, param in enumerate(learn.model.parameters()):
        if i<n: param.requires_grad=False

In [ ]:
learn.unfreeze()

In [ ]:
freeze_to(learn, 60)

In [ ]:
for i, param in enumerate(learn.model.parameters()):
    print(i,param.requires_grad)

In [ ]:
learn.opt.clear()

In [ ]:
learner.fit_one_cycle(2, max_lr=slice(0.00001), wd=0.01, pct_start=0.3)